In [ ]:
%%capture

%pip install openai nltk ipywidgets numpy requests-cache backoff

import os

# Sentiment Analysis with ChatGPT

Sentiment Analysis is sort of like the "Hello, world!" of Natural Language Processing (NLP), but luckily for us, it's a bit more fun than just echoing out a string - otherwise this workshop could be a bit bland.

This notebook will guide you through analyzing sentiment with ChatGPT and discuss some of the differences between how you can approach this problem with a generative AI like ChatGPT versus how you might have approached this problem in the past.

**Note**: For a better learning experience, this notebook purposely hides some implementation details like how interactive widgets are created and certain imports of notebook-specific utilities. Full details are available if you open this notebook in your editor of choice or expand the hidden cells.

## What is sentiment analysis?

Sentiment Analysis is a way of analyzing some text to determine if it's positive, negative, or neutral.

This is the kind of thing that's pretty easy for a human who understands the language the text is written in, but it can be hard for a computer to really understand the underlying meaning behind the text.

### Examples

1. "I saw that movie." - Neutral
2. "I love that movie." - Positive
3. "I hate that movie." - Negative

## How do we analyze sentiment?

We'll start with some housekeeping first by making sure that our dependencies are ready.

For this demo, we'll start out by exploring a more traditional approach that uses the Python Natural Language Toolkit (NLTK) and then we'll see how our approach might change when we use ChatGPT via the OpenAI SDK instead.

## Initial Setup

First, we'll import the relevant tools we'll be using in the notebook and configure some global variables.

In [6]:
import numpy as np
import nltk
import openai

nltk.download('vader_lexicon')

# globals
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TEMPERATURE = 0.2
STORY_SAMPLE_SIZE = 5

In [ ]:
import ipywidgets as pywidgets
import requests as request
import requests_cache
import backoff

# configuration widgets
from widgets.config import modelDropdown, apiKeyInput, apiKeyUpdateButton, temperatureSlider, sampleSizeSlider, sampleSizeWarningLabel, openAiHeader, hackerNewsHeader

# project-specific widgets
from widgets.simple import simpleAnalysisWidget
from widgets.advanced import advancedAnalysisWidget, configureOpenAi

# project-specific utilities
from utils.obfuscate import obfuscateKey

# globals
REQUEST_CACHE_EXPIRATION_SECONDS = 60 * 15

# we'll use this session to cache our hacker news api requests
session = requests_cache.CachedSession('hackernews_cache', expire_after=REQUEST_CACHE_EXPIRATION_SECONDS)

## Configuration

The code cell below renders a configuration form that you can use to adjust some variables used by other cells in this notebook.

You can make changes to the configuration form at any time and rerun cells that make requests to the OpenAI API or Hacker News API to see how the results change.

You can configure the following values:

- **Open AI API Key**: Your [OpenAI API key](https://platform.openai.com/account/api-keys) is read from the `$OPENAI_API_KEY` environment variable if it's set, but you can override it in this notebook. When you click the **Update Key** button the key you entered will be obfuscated and stored in the `OPENAI_API_KEY` global variable.
- **Model**: You can choose between the `gtp-3.5-turbo` and `gpt-4` models for this demo. The `gpt-4` model is more powerful, but it's also slower and more expensive to use.
- **Temperature**: A model's temperature is a measure of how "creative" or "unique" it's response will be. You can set this to `0` for something pretty close to deterministic responses to simple queries.
- **Sample Size**: We'll be gathering the top storeis from the [Hacker News API](https://github.com/HackerNews/API) and then sending the titles of a sample of those stories to the model for analysis. For quicker, cheaper results you may want to set this to a lower number. The larger your sample, the more tokens that will be consumed and the more likely you are to hit any rate limits

In [7]:
# this code cell is just used to display a widget
# for us to configure some settings that other cells
# in this notebook rely on
apiKeyInput.value = obfuscateKey(OPENAI_API_KEY)
sampleSizeSlider.value = STORY_SAMPLE_SIZE
temperatureSlider.value = TEMPERATURE

def updateApiKey(event):
  global OPENAI_API_KEY
  OPENAI_API_KEY = apiKeyInput.value
  apiKeyInput.value = obfuscateKey(OPENAI_API_KEY)

def updateSampleSize(change):
  global STORY_SAMPLE_SIZE
  STORY_SAMPLE_SIZE = change['new']

def updateTemperature(change):
  global TEMPERATURE
  TEMPERATURE = change['new']

temperatureSlider.observe(updateTemperature, names='value')
sampleSizeSlider.observe(updateSampleSize, names='value')
apiKeyUpdateButton.on_click(updateApiKey)

apiKeyConfigWidget = pywidgets.HBox([apiKeyInput, apiKeyUpdateButton])
openAiConfigWidget = pywidgets.VBox([openAiHeader, apiKeyConfigWidget, modelDropdown, temperatureSlider])
hackerNewsConfigWidget = pywidgets.VBox([hackerNewsHeader, sampleSizeSlider, sampleSizeWarningLabel])
configWidget = pywidgets.VBox([openAiConfigWidget, hackerNewsConfigWidget])

display(configWidget)

## Simple sentiment analysis with NLTK

Let's take a look at a simple example of sentiment analysis with `nltk` and VADER.

The `SentimentIntensityAnalyzer` returns an object with positive, negative, and neutral scores for the given text as well as a combined `compound` score computed from the other three.

For this basic example, we're going to rely on the `compound` score and use a naive rating scale.

In [8]:
# import the VADER sentiment analyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# instantiate the sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# analyze the sentiment of a string of text
def analyzeSentiment(text):
  if not text:
    return('')

  # use VADER to get the +/- sentiment of the string
  sentiment = analyzer.polarity_scores(text)

  # map the sentiment to a human readable label
  if sentiment['compound'] >= 0.75:
    return('Very Positive')
  elif sentiment['compound'] >= 0.4:
    return('Positive')
  elif sentiment['compound'] >= 0.1:
    return('Leaning Positive')
  elif sentiment['compound'] <= -0.1 and sentiment['compound'] > -0.4:
    return('Leaning Negative')
  elif sentiment['compound'] <= -0.4 and sentiment['compound'] > -0.75:
    return('Negative')
  elif sentiment['compound'] <= -0.75:
    return('Very Negative')
  else:
    return('Neutral')

Now let's test this analyzer with some example strings.

In [9]:
# some simple test statements for our analyzer
statements = [
  'I love that movie.',
  'I hate that movie.',
  'I like that movie.',
  'I dislike that movie.',
  'I saw that movie.',
]

for statement in statements:
  print(f"{statement} ({analyzeSentiment(statement)})")

I love that movie. (Positive)
I hate that movie. (Negative)
I like that movie. (Leaning Positive)
I dislike that movie. (Leaning Negative)
I saw that movie. (Neutral)


We've wired the input below up to the same analyzer function from above. Type in some text and see how the analyzer responds.

In [10]:
# this code cell is just used to display a widget
# that uses the analyzeSentiment function we created
display(simpleAnalysisWidget)

Box(children=(Text(value='', placeholder='Type something'), Output()), layout=Layout(align_items='center', dis…

## How Sentiment Analysis Works

Sentiment analysis, like most text analysis involves a multistep process:

1. **Stemming / Lemmatization**: reduces the words in the text to their root forms to simplify comparison between different forms of the same words
   1. **Stemming**: removes suffixes as an attempt to reduce words to their root forms
   2. **Lemmatization**: uses a morphological analysis of words to reduce them to their root forms
2. **Tokenization**: breaks the text into individual units of meaning called tokens
3. **Vectorization**: converts the tokens into a id that can be used for comparison
4. **Comparison**: compares the tokens to a known set of tokens to determine the sentiment

In this case we're taking advantage of an existing model that has been trained to analyze sentiment in text. If we wanted to build our own from scratch, it would be a more complicated process and require training data to feed into the model.

With the advent of Generative Pre-Trained Transformer (GPT) models like those that power ChatGPT, and other transformer models that have exploded in popularity since, we can leverage the powerful inference and predictive capabilities of these models to perform sentiment analysis without having to train our own model, and we can even leverage some prompting techniques to quickly teach the model how to perform more unique analyses.

## A more interesting example

So, let's see how this works with text generated by other humans without knowing that someone would be trying to analyze the sentiment of their text.

For this example, we'll pull in a random sample of the [top stories](https://github.com/HackerNews/API#new-top-and-best-stories) on [Hacker News](https://news.ycombinator.com/) and analyze the sentiment of each submission's title.

You can run the cell below a few times to generate different samples of the top stories until you find a collection you prefer and then rerun the cells below to use that sample for the rest of the notebook.

In [11]:
# we'll use this Array to aggregate the story titles so we can loop through and analyze them
stories = {}

# we'll use the request cache session we created earlier to make sure that this response is fast
# when the cell runs again - 
topStoryIdsRequest = session.get('https://hacker-news.firebaseio.com/v0/topstories.json')

if topStoryIdsRequest.status_code != 200:
  print('There was a problem getting the top stories from Hacker News')
  exit()

topStoryIds = topStoryIdsRequest.json()

storyIds = np.array(topStoryIds)[np.random.choice(len(topStoryIds), STORY_SAMPLE_SIZE, replace=False)]

for storyId in storyIds:
  # we'll use the same request cache so that we don't have to request a story's details more than once
  storyRequest = session.get(f'https://hacker-news.firebaseio.com/v0/item/{storyId}.json')

  if storyRequest.status_code != 200:
    continue
  else:
    story = storyRequest.json()

    if 'title' in story:
      stories[storyId] = {
        "title": story['title'],
        "time": story['time'],
        "sentiment": {
          "nltk": analyzeSentiment(story['title']),
          # we'll be updating the dict with the sentiment from OpenAI later
          "openai": {}
        }
      }

# iterate through titles and analyze the sentiment
for storyId, story in stories.items():
  print(f"{story['title']} ({story['sentiment']['nltk']})")

Clorox products in short supply after cyberattack (Neutral)
Americans Are Losing Faith in the Value of College. Whose Fault Is That? (Neutral)
Researchers gave 200 people $10k each to study generosity (Positive)
The path to detecting extraterrestrial life with astrophotonics (Neutral)
The Astrologer of the Nineteenth Century (1825) (Neutral)


## How ChatGPT works

Break down how ChatGPT turns text into tokens and then predicts the most likely tokens to follow the given text so far.

## Prompt engineering

Describe prompt engineering and break down system, user, and assistant prompts

## Zero shot and few shot prompting

Discuss the differences between few show and zero shot and give some examples

In [12]:
BASIC_SYSTEM_PROMPT = """
You are VibeCheck, an advanced AI system for detecting the sentiment conveyed in user-generated text.

The user will provide you with a prompt, and you will respond with the sentiment of that prompt.

Be concise.
"""

@backoff.on_exception(backoff.expo, openai.error.RateLimitError)
def basicChatGptSentiment(prompt, model=modelDropdown.value):
    messages = [{ "role": "system", "content": BASIC_SYSTEM_PROMPT }]

    messages.append({"role": "user", "content": prompt})

    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )

    return response.choices[0].message["content"]

In [13]:
if OPENAI_API_KEY:
  for storyId, story in stories.items():
    sentiment = basicChatGptSentiment(story['title'])
    
    if modelDropdown.value not in story['sentiment']['openai']:
      story['sentiment']['openai'][modelDropdown.value] = {}

    story['sentiment']['openai'][modelDropdown.value]['basic'] = sentiment

    print(f"{story['title']}\nNLTK: {story['sentiment']['nltk']}\n{modelDropdown.value}: {sentiment}\n---")
else:
  print('Please enter your OpenAI API key above and rerun this cell')

Clorox products in short supply after cyberattack
NLTK: Neutral
gpt-3.5-turbo: Neutral
---
Americans Are Losing Faith in the Value of College. Whose Fault Is That?
NLTK: Neutral
gpt-3.5-turbo: Neutral
---
Researchers gave 200 people $10k each to study generosity
NLTK: Positive
gpt-3.5-turbo: Positive
---
The path to detecting extraterrestrial life with astrophotonics
NLTK: Neutral
gpt-3.5-turbo: Exciting and optimistic.
---
The Astrologer of the Nineteenth Century (1825)
NLTK: Neutral
gpt-3.5-turbo: Neutral
---


In [14]:
ADVANCED_SYSTEM_PROMPT = """
You are VibeCheck, an advanced AI system for detecting the sentiment conveyed in user-generated text.

The user will provide you with a prompt, and you will analyze it following these steps:

1. Analyze the prompt for relevant emotion, tone, affinity, sarcasm, irony, etc.
2. Analyze the likely emotional state of the author based on those findings
3. Summarize the emotional state and sentiment of the prompt based on your findings using 5 or less names for emotions using lowercase letters and separating each emotional state with a comma

Only return the output from the final step to the user.
"""

@backoff.on_exception(backoff.expo, openai.error.RateLimitError)
def advancedChatGptSentiment(prompt, model=modelDropdown.value):
    messages = [{ "role": "system", "content": ADVANCED_SYSTEM_PROMPT }]

    messages.append({"role": "user", "content": prompt})

    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0.25,
    )

    return response.choices[0].message["content"]

In [15]:
if OPENAI_API_KEY:
  for storyId, story in stories.items():
    sentiment = advancedChatGptSentiment(story['title'])

    if modelDropdown.value not in story['sentiment']['openai']:
      story['sentiment']['openai'][modelDropdown.value] = {}

    story['sentiment']['openai'][modelDropdown.value]['advanced'] = sentiment

    print(f"{story['title']}\nNLTK: {story['sentiment']['nltk']}\n{modelDropdown.value}: {sentiment}\n---")
else:
  print('Please enter your OpenAI API key above and rerun this cell')

Clorox products in short supply after cyberattack
NLTK: Neutral
gpt-3.5-turbo: concerned, frustrated, anxious
---
Americans Are Losing Faith in the Value of College. Whose Fault Is That?
NLTK: Neutral
gpt-3.5-turbo: uncertainty, frustration, blame, skepticism
---
Researchers gave 200 people $10k each to study generosity
NLTK: Positive
gpt-3.5-turbo: neutral, curious, hopeful
---
The path to detecting extraterrestrial life with astrophotonics
NLTK: Neutral
gpt-3.5-turbo: curiosity, excitement, anticipation
---
The Astrologer of the Nineteenth Century (1825)
NLTK: Neutral
gpt-3.5-turbo: curiosity, intrigue, fascination
---


In [16]:
# this code cell is just used to display a widget
# that uses the analyzeSentiment function we created
# as well as the advancedChatGptSentiment function
configureOpenAi(OPENAI_API_KEY, modelDropdown.value)

display(advancedAnalysisWidget)